In [1]:
import tensorflow as tf
import numpy as np
import os

Параметры

In [ ]:
SEQ_LENGTH = 100  
BATCH_SIZE = 64
BUFFER_SIZE = 10000
EMBED_DIM = 256
LSTM_UNITS = 512
EPOCHS = 60

In [11]:
path_to_file = "text.txt"
text = open(path_to_file, "r", encoding="utf-8").read()
print(f"Total characters: {len(text)}")

vocab = sorted(set(text))
print(f"Unique characters: {len(vocab)}")
char2idx = {c:i for i,c in enumerate(vocab)}
idx2char = np.array(vocab)

Total characters: 75259
Unique characters: 76


Подготовка данных

In [ ]:
text_as_int = np.array([char2idx[c] for c in text])

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(SEQ_LENGTH+1, drop_remainder=True)

def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

dataset = sequences.map(split_input_target)

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
)


Построение модели

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(vocab), EMBED_DIM),
    tf.keras.layers.LSTM(LSTM_UNITS,
                         return_sequences=True,
                         stateful=False),
    tf.keras.layers.Dense(len(vocab))
])

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)
model.fit(dataset, epochs=1)
model.summary()

11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 467ms/step - loss: 3.8334


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (64, 100, 256)         │        19,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (64, 100, 512)         │     1,574,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (64, 100, 76)          │        38,988 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,900,070 (18.69 MB)

 Trainable params: 1,633,356 (6.23 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,266,714 (12.46 MB)

обучение модели

In [14]:
print("Training...")
model.fit(dataset, epochs=EPOCHS)

Training...
Epoch 1/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 516ms/step - loss: 3.4198
Epoch 2/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 585ms/step - loss: 3.2740
Epoch 3/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 528ms/step - loss: 3.0451
Epoch 4/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 489ms/step - loss: 2.8359
Epoch 5/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 477ms/step - loss: 2.6928
Epoch 6/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 472ms/step - loss: 2.6140
Epoch 7/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 566ms/step - loss: 2.5642
Epoch 8/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 511ms/step - loss: 2.5252
Epoch 9/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 497ms/step - loss: 2.4904
Epoch 10/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 483ms/step - loss: 2.4702
Epoch 11/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 478ms/step - loss: 2.4334
Epoch 12/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 539ms/step - loss: 2.4086
Epoch 13/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 513ms/step - loss: 2.3814
Epoch 14/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 476ms/step - loss: 2.3579
Epoch 15/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 519

In [18]:
def generate_text(model, start_string, num_generate=1000, temperature=1.0):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = predictions[:, -1, :] / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    with open("generated_text.txt", "w", encoding="utf-8") as f:
        f.write(start_string + "".join(text_generated))
    return start_string + "".join(text_generated)


seed = "Для вас, души "
print("Generated text:\n")
print(generate_text(model, seed))

Generated text:

Для вас, души су корь меепочь оважедуплетв о ствонынублый воживездизй…, пеегисжюбо м мнит, ныМрь, блазгобубу у гая маря гнаки кру,
Скися,
Тыт тязалОсё!жы мчудетой кинанелсыне чет кжелелс сновит;
Пла раретот м, ти жи ве тьм м пла нусё гейВлеслепол многры, свс чубя еляствоомемеметечигаОст бы.
Нарамира пь хоюмерилавеных, Вво ви;
Туя грше,
Куног
Нвипры…Лит;
«Я босною нои вн;
Навечеразяз-оком;
Та»Цноты;
Дрыть,
Пови к
«?Пх:Жечома:)ни,;
НанегЛДихигдой;
Пабекх)«Упокожидря му стесы ЛСтореза,
См Нола у то тьсаля модесь уялазы, встрко:Щю
НогихЖо пей гох!Тя ся ноноврянЯ мнопрози отврамарезь повлаля!
Пою. м!.
Отли нуюбом смеласл
МящесуЖудодля равоюм?чегожезал и;
И взно лы!».
Вхоли, тре х,
Угнокизв ла-г:, нидуст лах;
Поднолсмязапь ув бети….
«Хыста……Ты чь орависть лить?И се,
Флмотвоу нилим оцЖитрезаме пУ в Лю бой дносу тлы, цИ норажилИ водувьчитодафитидуга в сста се.
в?-стныхбелун обленопоскодоля м й виле сис у боготаю пи г,
Тотв ко Кидмода весогимс
И сЛжисть:Гли, ть!»)две ф мнылелел